### Sets

In [ ]:
tickers = ['AAPL', 'MSFT', 'GOOG','SAP', 'RGTI', 'TSLA', 'GLD','BTC']

# Initialize Pyomo model
model = pyo.ConcreteModel(name='Financial_Portfolio')
model.Assets = pyo.Set(initialize=tickers)

### Parameters




In [ ]:
'''LOCAL PARAMETERS'''

#1 returns of the set of assets
data = yf.download(tickers, start='2019-12-31', end='2024-12-31')#using API to obtain historical data of closing prices
closing_daily_prices = data["Close"] # Extract close prices to compute daily returns in percentages in the historical horizon
returns = closing_daily_prices.pct_change().dropna() # daily rate of change return
mean_daily_returns = returns.mean().values
mean_daily_returns=np.squeeze(mean_daily_returns)
mean_daily_returns=np.round(mean_daily_returns, 3)

#2 cost: closing price of today 02-01-2025
cost1=(yf.download(tickers, start='2025-01-02', end='2025-01-03'))['Close']
costs=np.squeeze(cost1)
costs=round(costs, 3)

#3 risk : variance  and covariance matrix
cov_matrix = returns.cov().values

#4 lminall lmaxall
Lminall = [0.05, 0.02, 0.03, 0.1, 0.15, 0.05, 0.01, 0.02]
Lmaxall = [0.8, 0.7, 0.9, 0.6, 0.85, 0.9, 0.7, 0.9]

'''GLOBAL PARAMETERS'''
B= 3000 #5 capital budget
Riskmax= 990  #6 riskmax
K=2 #min num of asset must be equal to the number of a
alpha= 0.9 #propenso al rischio
parameters = {
    'returns': {tickers[i]: mean_daily_returns[i] for i in range(len(tickers))},
    'costs': {tickers[i]: costs[i] for i in range(len(tickers))},
    'riskCovariance': {(tickers[i], tickers[j]): cov_matrix[i, j] for i in range(len(tickers)) for j in range(len(tickers))},
    'L_min_allocations': {tickers[i]: Lminall[i] for i in range(len(tickers))},
    'U_max_allocations': {tickers[i]: Lmaxall[i] for i in range(len(tickers))},
    'Budget': B,
    'Risk_Max': Riskmax,
    'Min_Assets': K,
    'Alpha': alpha,}

# local parameters to the model
model.returns = pyo.Param(model.Assets, initialize=parameters['returns'])
model.costs = pyo.Param(model.Assets, initialize=parameters['costs'])
model.L_min_allocations = pyo.Param(model.Assets, initialize=parameters['L_min_allocations'])
model.U_max_allocations = pyo.Param(model.Assets, initialize=parameters['U_max_allocations'])

# compute MArkovitz risk measure
model.riskCovariance = pyo.Param(model.Assets, model.Assets, initialize=parameters['riskCovariance'])
# Compute MAD risk measure
mad_risk = returns.sub(returns.mean()).abs().mean().values
model.madRisk = pyo.Param(model.Assets, initialize={tickers[i]: mad_risk[i] for i in range(len(tickers))})

#scalar parameters to the model
model.Budget = pyo.Param(initialize=parameters['Budget'])
model.Risk_Max = pyo.Param(initialize=parameters['Risk_Max'])
model.Min_Assets = pyo.Param(initialize=parameters['Min_Assets'])
model.Alpha = pyo.Param(initialize=parameters['Alpha'])
print('\n\n')

### Variables




In [ ]:
#Continuous variable with 0 , 1 bounds
model.x = pyo.Var(model.Assets, within=pyo.NonNegativeReals, bounds=(0, 1))

#Binary Variable
model.y=pyo.Var(model.Assets, within=pyo.Binary)

### Constraints



In [ ]:
def budget_constraint(model): #BUDGET
    return sum(model.costs[i] * model.x[i] for i in model.Assets) <= model.Budget
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def total_b(model): #CAPITAL
  return sum(model.x[i] for i in model.Assets)==1
model.total_b=pyo.Constraint(rule=total_b)

def diversificationk(model): #DIVERSIFICATION K
  return sum(model.y[i] for i in model.Assets) >= model.Min_Assets
model.diversificationk=pyo.Constraint(rule=diversificationk)
#model.diversification_constraint = pyo.Constraint(expr=sum(model.y[i] for i in model.Assets) >= model.Min_Assets)

def purchasing_constraint(model, i): #list of contraints
    return model.x[i] <= model.y[i]
model.purchasing_constraint = pyo.Constraint(model.Assets, rule=purchasing_constraint)
'''
model.purchasing_constraints = pyo.ConstraintList()
for i in model.Assets:
    model.purchasing_constraints.add(model.x[i] <= model.y[i])
'''
def min_allocation(model, i): #L Min
    return model.x[i] >= model.y[i]*model.L_min_allocations[i]  # Deve essere >= e non >
model.min_allocation_constr = pyo.Constraint(model.Assets, rule=min_allocation)

def max_allocation(model,i): #U Max
  return model.x[i] <= model.y[i]*model.U_max_allocations[i]
model.max_allocation_constr = pyo.Constraint(model.Assets, rule=max_allocation)
'''
model.investment_constraints = pyo.ConstraintList()
for i in model.Assets:
    model.investment_constraints.add(model.y[i] * model.L_min_allocations[i] <= model.x[i])
    model.investment_constraints.add(model.x[i] <= model.y[i] * model.U_max_allocations[i])
'''

# MAD RISK  constraints
model.risk_constraint_MAD = pyo.Constraint(expr=sum(model.x[i] * model.madRisk[i] for i in model.Assets) <= model.Risk_Max)

model.binaryConstraint= pyo.Constraint(expr=(model.y['BTC']+model.y['TSLA'])<=1)

### Objective function

In [ ]:
# Objective Function using MAD risk measure
model.obj = pyo.Objective(
    expr=(model.Alpha*sum(model.x[i] * model.returns[i] for i in model.Assets)) -
         ((1 - model.Alpha) * sum(model.x[i] * model.madRisk[i] for i in model.Assets)),
    sense=pyo.maximize)

## Model solution

In [ ]:
# Solve the model MAD
solver = pyo.SolverFactory('cbc')
results=solver.solve(model)

print(results.solver.termination_condition)
print(results.solver.status)

print("=== RESULTS WITH MAD RISK===")
for i in model.Assets:
  if model.y[i].value==0:
    continue
  print(f' X {i} : {pyo.value(model.x[i])}, Y : {pyo.value(model.y[i])}')

optimal
ok
=== RESULTS WITH MAD RISK===
 X MSFT : 0.1, Y : 1.0
 X TSLA : 0.9, Y : 1.0


In [ ]:
# Dictionary to store results
dict_of_results_MAD = {}
selected_assets = {i: model.y[i].value for i in model.Assets}
fractional_investments = {i: model.x[i].value for i in model.Assets}
portfolio_return = sum(model.x[i] * model.returns[i] for i in model.Assets)
portfolio_risk = sum(model.x[i] * model.madRisk[i] for i in model.Assets)

from pyomo.environ import value

# Extracting and printing the values of the objective function, portfolio return, and portfolio risk
objective_value = value(model.obj)
portfolio_return_value = value(portfolio_return)
portfolio_risk_value = value(portfolio_risk)
dict_of_results_MAD = {
        "objective_value": objective_value,
        "portfolio_return": portfolio_return_value,
        "portfolio_risk": portfolio_risk_value,
        "selected_assets_Y": selected_assets,
        "x_fractional_investments_X": fractional_investments}
pprint(dict_of_results_MAD)


{'objective_value': np.float64(0.019769759189806967),
 'portfolio_return': np.float64(0.031000000000000003),
 'portfolio_risk': np.float64(0.08130240810193042),
 'selected_assets_Y': {'AAPL': 0.0,
                       'BTC': 0.0,
                       'GLD': 0.0,
                       'GOOG': 0.0,
                       'MSFT': 1.0,
                       'RGTI': 0.0,
                       'SAP': 0.0,
                       'TSLA': 1.0},
 'x_fractional_investments_X': {'AAPL': 0.0,
                                'BTC': 0.0,
                                'GLD': 0.0,
                                'GOOG': 0.0,
                                'MSFT': 0.1,
                                'RGTI': 0.0,
                                'SAP': 0.0,
                                'TSLA': 0.9}}


## 5.3 - Analysis



In [ ]:
# Assuming model has been defined and solver is set up

alpha_values = [0, 0.2, 0.4, 0.6, 0.8, 1.0]  # Values for alpha
results = []  # To store the results for each alpha

# Loop over each alpha value
for alpha in alpha_values:
    # Update the model's objective function based on the current value of alpha
    model.obj = pyo.Objective(
        expr=(alpha * sum(model.x[i] * model.returns[i] for i in model.Assets)) -
             ((1 - alpha) * sum(model.x[i] * model.madRisk[i] for i in model.Assets)),
        sense=pyo.maximize
    )

    # Solve the model
    solver = SolverFactory('cbc')
    solver.solve(model, tee=False)

    # Collect the results: Total return, total risk, and objective value
    total_return = sum(model.x[i].value * model.returns[i] for i in model.Assets)
    total_risk = sum(model.x[i].value * model.madRisk[i] for i in model.Assets)
    total_obj_value = value(model.obj)  # Get the objective value

    # Store the results
    results.append({
        'Alpha': alpha,
        'Total Return': total_return,
        'Total Risk (MAD)': total_risk,
        'Objective Value': total_obj_value
    })
print('\n\n\n')
print('=====================Results=====================')
pprint(results)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().






=====================Results=====================
[{'Alpha': 0,
  'Objective Value': np.float64(-0.00754607447122204),
  'Total Return': np.float64(0.001),
  'Total Risk (MAD)': np.float64(0.00754607447122204)},
 {'Alpha': 0.2,
  'Objective Value': np.float64(-0.005836859576977633),
  'Total Return': np.float64(0.001),
  'Total Risk (MAD)': np.float64(0.00754607447122204)},
 {'Alpha': 0.4,
  'Objective Value': np.float64(-0.004127644682733224),
  'Total Return': np.float64(0.001),
  'Total Risk (MAD)': np.float64(0.00754607447122204)},
 {'Alpha': 0.6,
  'Objective Value': np.float64(-0.0024184297884888165),
  'Total Return': np.float64(0.001),
  'Total Risk (MAD)': np.float64(0.00754607447122204)},
 {'Alpha': 0.8,
  'Objective Value': np.float64(0.00872210609818325),
  'Total Return': np.float64(0.0308),
  'Total Risk (MAD)': np.float64(0.07958946950908377)},
 {'Alpha': 1.0,
  'Objective Value': np.float64(0.031000000000000003),
  'Total Return': np.float64(0.031000000000000003),
 

## 5.2 - Model solution : with Markovitz



In [ ]:
#Continuous variable with 0 , 1 bounds
model.x = pyo.Var(model.Assets, within=pyo.NonNegativeReals, bounds=(0, 1))
#Binary Variable
model.y=pyo.Var(model.Assets, within=pyo.Binary)
#for markovitz model
model.z=pyo.Var(model.Assets, model.Assets, within=pyo.NonNegativeReals, bounds=(0, 1))

#CONSTRAINTS

def budget_constraint(model): #BUDGET
    return sum(model.costs[i] * model.x[i] for i in model.Assets) <= model.Budget
model.budget_constraint = pyo.Constraint(rule=budget_constraint)

def total_b(model): #CAPITAL
  return sum(model.x[i] for i in model.Assets)==1
model.total_b=pyo.Constraint(rule=total_b)

def diversificationk(model): #DIVERSIFICATION K
  return sum(model.y[i] for i in model.Assets) >= model.Min_Assets
model.diversificationk=pyo.Constraint(rule=diversificationk)
#model.diversification_constraint = pyo.Constraint(expr=sum(model.y[i] for i in model.Assets) >= model.Min_Assets)

def purchasing_constraint(model, i): #list of contraints
    return model.x[i] <= model.y[i]
model.purchasing_constraint = pyo.Constraint(model.Assets, rule=purchasing_constraint)
'''
model.purchasing_constraints = pyo.ConstraintList()
for i in model.Assets:
    model.purchasing_constraints.add(model.x[i] <= model.y[i])
'''
def min_allocation(model, i): #L Min
    return model.x[i] >= model.y[i]*model.L_min_allocations[i]  # Deve essere >= e non >
model.min_allocation_constr = pyo.Constraint(model.Assets, rule=min_allocation)

def max_allocation(model,i): #U Max
  return model.x[i] <= model.y[i]*model.U_max_allocations[i]
model.max_allocation_constr = pyo.Constraint(model.Assets, rule=max_allocation)
'''
model.investment_constraints = pyo.ConstraintList()
for i in model.Assets:
    model.investment_constraints.add(model.y[i] * model.L_min_allocations[i] <= model.x[i])
    model.investment_constraints.add(model.x[i] <= model.y[i] * model.U_max_allocations[i])
'''

#MARKOVITZ CONSTRAINT
def Mark_risk_contraint (model):
    return sum(model.z[i, j]*model.riskCovariance[i,j] for i in model.Assets for j in model.Assets) <= model.Risk_Max
model.risk_constraint_Mark = pyo.Constraint(rule=Mark_risk_contraint)

#MODELLO 1 Markovitz with linearization of xixy
#  variabiles' constraints
def mccormick1(model, i, j):
    return model.z[i, j] >= model.x[i] + model.x[j] - 1

def mccormick2(model, i, j):
    return model.z[i, j] <= model.x[i]

def mccormick3(model, i, j):
    return model.z[i, j] <= model.x[j]

def mccormick4(model, i, j):
    return model.z[i, j] >= 0

model.mccormick1 = pyo.Constraint(model.Assets, model.Assets, rule=mccormick1)
model.mccormick2 = pyo.Constraint(model.Assets, model.Assets, rule=mccormick2)
model.mccormick3 = pyo.Constraint(model.Assets, model.Assets, rule=mccormick3)
model.mccormick4 = pyo.Constraint(model.Assets, model.Assets, rule=mccormick4)


#OBJECTIVE MARKOVITZ
def total_return(model):
  tot_return=sum(model.returns[i] * model.x[i]  for i in model.Assets)
  tot_risk=sum(model.z[i,j]*model.riskCovariance[i,j] for i in model.Assets for j in model.Assets)
  return tot_return*model.Alpha - (1-model.Alpha)*tot_risk
model.obj = pyo.Objective(rule= total_return, sense=pyo.maximize)


print("=== RESULTS ===")
#model.display()
solver = pyo.SolverFactory('cbc')
results=solver.solve(model, tee=False)
print(results.solver.termination_condition)
print(results.solver.status)

for i in model.Assets:
  if model.y[i].value==0:
    continue
  print(f' X {i} : {pyo.value(model.x[i])}, Y : {pyo.value(model.y[i])}')

for i in model.Assets:
  for j in model.Assets:
    if model.z[i,j].value==0:
      continue
    print(f"z {i,j}: {model.z[i,j].value}")

for i in model.Assets:
  for j in model.Assets:
    if model.x[i].value*model.x[j].value==0:
      continue
    print(f"x{i, j}: {model.x[i].value, model.x[j].value, model.x[i].value*model.x[j].value}")

print('\n\n')
#print('dev std con z:', sqrt(0.75*0.015936055903302365))


dict_of_results_Mark = {}
selected_assets_Mark = {i: model.y[i].value for i in model.Assets}
fractional_investments_Mark = {i: model.x[i].value for i in model.Assets}
portfolio_return_Mark = sum(fractional_investments[i] * parameters['returns'][i] for i in tickers)
portfolio_risk_Mark = sum(model.x[i] * model.madRisk[i] for i in model.Assets)
dict_of_results_Mark = {
        "objective_value": pyo.value(model.obj),
        "portfolio_return": portfolio_return_Mark,
        "portfolio_risk": value(portfolio_risk_Mark),
        "selected_assets_Y": selected_assets_Mark,
        "x_fractional_investments_X": fractional_investments_Mark}
pprint(dict_of_results_Mark)

